In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import os
import re
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import string
from collections import defaultdict
from nltk.stem.porter import PorterStemmer
import pickle
from IPython.display import clear_output
from gensim import corpora, models
from gensim.models import Phrases, Word2Vec
from gensim.models.phrases import Phraser
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
funded_R01_projects_unique_dropped = pd.read_csv("data/funded_R01_projects_category.csv")

/Users/maheshrao/anaconda3/envs/capstone/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,12,13,16,20,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
funded_R01_projects_unique_dropped.head()

,Unnamed: 0,APPLICATION_ID,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_END,BUDGET_START,CFDA_CODE,CORE_PROJECT_NUM,DIRECT_COST_AMT,ED_INST_TYPE,...,SUFFIX,TOTAL_COST,TOTAL_COST_SUB_PROJECT,ABSTRACT_TEXT,grant_num,topics_proj_terms,topics_spending_cats,topics_total,category,probability
0,2185,7499670,NaN,2008-09-12T00:00:00,08/31/2009,09/01/2008,389.0,U24RR018107,NaN,SCHOOLS OF MEDICINE,...,NaN,741430.0,NaN,[unreadable] DESCRIPTION (provided by applic...,U24RR018107,[],[],[],Tuberculosis,1.000000
1,2186,7495989,NaN,2008-09-16T00:00:00,06/30/2009,09/01/2008,389.0,U24RR018108,NaN,UNIVERSITY-WIDE,...,NaN,1811959.0,NaN,[unreadable] DESCRIPTION (provided by applic...,U24RR018108,[],[],[],Tuberculosis,0.999931
2,2187,7502303,NaN,2008-09-01T00:00:00,06/30/2009,09/01/2008,389.0,U24RR018109,NaN,PRIMATE CENTERS,...,A1,1431792.0,NaN,[unreadable] DESCRIPTION (provided by applic...,U24RR018109,[],[],[],Tuberculosis,0.999549
3,2188,7501548,NaN,2008-06-30T00:00:00,06/30/2009,07/01/2008,389.0,U24RR018111,NaN,PRIMATE CENTERS,...,A1,1908805.0,NaN,[unreadable] DESCRIPTION (provided by applic...,U24RR018111,[],[],[],Tuberculosis,1.000000
4,2189,7439671,NaN,2008-08-16T00:00:00,05/31/2009,08/18/2008,389.0,U24RR018144,NaN,PRIMATE CENTERS,...,NaN,2067808.0,NaN,[unreadable] DESCRIPTION (provided by applic...,U24RR018144,[],[],[],Tuberculosis,0.999975


In [4]:
len(funded_R01_projects_unique_dropped)

272098

In [5]:
funded_projects = pd.DataFrame()
for root, dirs, files in os.walk("data/completed_full_csv_parts/", topdown=True):
    for file in files:
        df = pd.read_csv(os.path.join(root, file))
        funded_projects = funded_projects.append(df)

/Users/maheshrao/anaconda3/envs/capstone/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,5,11,12,15,17,19,23,24,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/maheshrao/anaconda3/envs/capstone/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,12,19,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/maheshrao/anaconda3/envs/capstone/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,11,12,15,19,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/maheshrao/anaconda3/envs/capstone/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (

In [6]:
funded_R01_projects = funded_projects[funded_projects.grant_num.str.contains("R01")]

In [7]:
funded_R01_projects_complete = funded_R01_projects.merge(funded_R01_projects_unique_dropped[["ABSTRACT_TEXT", "category", "probability"]], how = "left", on = "ABSTRACT_TEXT")

In [8]:
print(len(funded_R01_projects), len(funded_R01_projects_unique_dropped), len(funded_R01_projects_complete))

645439 272098 645439


In [12]:
funded_R01_projects_complete

,APPLICATION_ID,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_END,BUDGET_START,CFDA_CODE,CORE_PROJECT_NUM,DIRECT_COST_AMT,ED_INST_TYPE,FOA_NUMBER,...,PROJECT_TITLE,STUDY_SECTION,SUBPROJECT_ID,SUFFIX,TOTAL_COST,TOTAL_COST_SUB_PROJECT,ABSTRACT_TEXT,grant_num,category,probability
0,7499670,NaN,2008-09-12T00:00:00,08/31/2009,09/01/2008,389.0,U24RR018107,NaN,SCHOOLS OF MEDICINE,NaN,...,ESTABLISHMENT OF SPECIFIC PATHOGEN FREE RHESUS...,RIRG,NaN,NaN,741430.0,NaN,[unreadable] DESCRIPTION (provided by applic...,U24RR018107,Tuberculosis,1.000000
1,7495989,NaN,2008-09-16T00:00:00,06/30/2009,09/01/2008,389.0,U24RR018108,NaN,UNIVERSITY-WIDE,NaN,...,ENHANCEMENT OF THE CPRC-SPF RHESUS MONKEY PROGRAM,RIRG,NaN,NaN,1811959.0,NaN,[unreadable] DESCRIPTION (provided by applic...,U24RR018108,Tuberculosis,0.999931
2,7502303,NaN,2008-09-01T00:00:00,06/30/2009,09/01/2008,389.0,U24RR018109,NaN,PRIMATE CENTERS,NaN,...,SPF Breeding Colonies at the Yerkes NPRC,RIRG,NaN,A1,1431792.0,NaN,[unreadable] DESCRIPTION (provided by applic...,U24RR018109,Tuberculosis,0.999549
3,7501548,NaN,2008-06-30T00:00:00,06/30/2009,07/01/2008,389.0,U24RR018111,NaN,PRIMATE CENTERS,NaN,...,Establishment and Expansion of SPF Rhesus Colony,RIRG,NaN,A1,1908805.0,NaN,[unreadable] DESCRIPTION (provided by applic...,U24RR018111,Tuberculosis,1.000000
4,7439671,NaN,2008-08-16T00:00:00,05/31/2009,08/18/2008,389.0,U24RR018144,NaN,PRIMATE CENTERS,NaN,...,Genetically Defined Herpes/Retrovirus SPF Maca...,RIRG,NaN,NaN,2067808.0,NaN,[unreadable] DESCRIPTION (provided by applic...,U24RR018144,Tuberculosis,0.999975
5,7417544,NaN,2008-04-30T00:00:00,04/30/2010,05/01/2008,389.0,U24RR019701,NaN,SCHOOLS OF MEDICINE,NaN,...,BIOMEDICAL IMAGING RESEARCH NETWORK,ZRR1,NaN,NaN,3672199.0,NaN,DESCRIPTION (provided by applicant): The Bi...,U24RR019701,Schizophrenia,0.999992
6,7667600,NaN,2008-09-16T00:00:00,04/30/2010,09/20/2008,389.0,U24RR019701,NaN,SCHOOLS OF MEDICINE,NaN,...,BIOMEDICAL IMAGING RESEARCH NETWORK,ZRR1,NaN,S1,160000.0,NaN,DESCRIPTION (provided by applicant): The Bi...,U24RR019701,Schizophrenia,0.999992
7,7725973,NaN,2008-04-30T00:00:00,04/30/2009,05/01/2008,NaN,U24RR019701,NaN,SCHOOLS OF MEDICINE,NaN,...,BIOMEDICAL INFORMATICS RESEARCH NETWORK: NEURO...,ZRR1,7592.0,NaN,NaN,1836100.0,DESCRIPTION (provided by applicant): The Bi...,U24RR019701,Schizophrenia,0.999992
8,7725974,NaN,2008-04-30T00:00:00,04/30/2009,05/01/2008,NaN,U24RR019701,NaN,SCHOOLS OF MEDICINE,NaN,...,BIOMEDICAL INFORMATICS RESEARCH NETWORK: NATIO...,ZRR1,7593.0,NaN,NaN,160000.0,DESCRIPTION (provided by applicant): The Bi...,U24RR019701,Schizophrenia,0.999992
9,7725975,NaN,2008-04-30T00:00:00,04/30/2009,05/01/2008,NaN,U24RR019701,NaN,SCHOOLS OF MEDICINE,NaN,...,BIOMEDICAL IMAGING RESEARCH NETWORK: NEUROLOGI...,ZRR1,7594.0,NaN,NaN,1836099.0,DESCRIPTION (provided by applicant): The Bi...,U24RR019701,Schizophrenia,0.999992


In [14]:
funded_R01_projects_complete.to_csv("funded_R01_projects_with_categories.csv")

OSError: [Errno 22] Invalid argument

In [18]:
funded_R01_projects_complete.to_csv("data/funded_R01_projects_category.csv", chunksize = 1000)

OSError: [Errno 22] Invalid argument

In [17]:
funded_R01_projects_complete.to_pickle("data/funded_R01_projects_category.pkl")

OSError: [Errno 22] Invalid argument

In [19]:
[funded_R01_projects_complete_1, funded_R01_projects_complete_2] = np.array_split(funded_R01_projects_complete, 2)

In [20]:
df_list = [funded_R01_projects_complete_1, funded_R01_projects_complete_2]
df_names = ["funded_R01_projects_complete_1", "funded_R01_projects_complete_2"]

In [21]:
for name, df in zip(df_names, df_list):
    df.to_csv('data/{}.csv'.format(name), index = False)